In [1]:
import os
import shutil
# 這兩個套件是為了要將訓練資料的路徑讀取工具也引進來

os.environ['TF_USE_LEGACY_KERAS'] = '1'

import tensorflow as tf # 就是 Tensorflow 不用多說
import tensorflow_hub as hub # Tensorflow bert 社群
import tensorflow_text as text # 前處理
from official.nlp import optimization  # 優化工具

import matplotlib.pyplot as plt # 畫圖用的套件

tf.get_logger().setLevel('ERROR') # Debug的工具，也可以不寫.


2025-01-14 20:19:30.958821: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-14 20:19:30.969075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736857170.984791   12456 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736857170.990274   12456 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 20:19:31.006246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
    

# with tf.Session() as sess:
#     print (sess.run(c))

I0000 00:00:1736857173.147195   12456 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1401 MB memory:  -> device: 0, name: NVIDIA GeForce MX550, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
assert tf.test.is_gpu_available()
assert tf.test.is_built_with_cuda()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8888880915860543667
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1469382656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9903331169386631404
physical_device_desc: "device: 0, name: NVIDIA GeForce MX550, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


I0000 00:00:1736857173.220872   12456 gpu_device.cc:2022] Created device /device:GPU:0 with 1401 MB memory:  -> device: 0, name: NVIDIA GeForce MX550, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1736857173.228561   12456 gpu_device.cc:2022] Created device /device:GPU:0 with 1401 MB memory:  -> device: 0, name: NVIDIA GeForce MX550, pci bus id: 0000:01:00.0, compute capability: 7.5


In [4]:
from core.general import *

info(tf.__version__)
tf.test.is_gpu_available()

INFO  2.18.0


I0000 00:00:1736857173.241678   12456 gpu_device.cc:2022] Created device /device:GPU:0 with 1401 MB memory:  -> device: 0, name: NVIDIA GeForce MX550, pci bus id: 0000:01:00.0, compute capability: 7.5


True

In [5]:
# # 這個是資料集的下載連結，如果你複製這段 url 到瀏覽器上，也可以下載的到
# url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

# dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
#                                   untar=True, cache_dir='.',
#                                   cache_subdir='')

# # 所謂的 os.path.join 就是把兩個路徑結合起來的意思，如果路徑是 ./data/，join之後就會變成 ./data/aclImdb，而以下所做的，就是要整合資料路徑，方便之後模型讀取
# dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

# train_dir = os.path.join(dataset_dir, 'train')

# # remove unused folders to make it easier to load the data
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)


In [6]:
from core.general import *
ls('../data')

import pandas as pd
f = pd.read_csv('../data/filtered_train_data.csv')
f = f[['Statement', 'tags']]
f['isDP'] = ['dp' in i for i in f['tags'].tolist()]
f = f.drop('tags', axis = 1)
f

,Statement,isDP
0,watch new seri squid game mashtali soroush dec...,False
1,today math lesson teacher told vovochka euler ...,False
2,sequenc integ call good max min sequenc integ ...,True
3,petya got interest grammar third year school i...,False
4,nezzar design brand new game hidden permut sha...,False
...,...,...
7971,given array integ find maximum possibl valu am...,True
7972,polycarp poor memori day rememb 3 differ lette...,False
7973,hard version problem differ easi version versi...,False
7974,two chess piec rook knight stand standard ches...,False


In [7]:
pre = f.to_dict()
# for i in train:
# debug(pwd())
cd(root)
cd('aclImdb/train')
for tag in ['pos', True], ['neg', False]:
    cd(tag[0])
    for i in range(len(f['Statement'])):
        if pre['isDP'][i] ^ tag[1]: continue;
        write_to_file(f'{i}.txt', pre['Statement'][i])
    cd('..')
cd('../..')

In [8]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

info('Loading Training Data.')
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

info('Loading Validating Data.')
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

info('Loading Testing Data.')
test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


INFO  Loading Training Data.
Found 7976 files belonging to 2 classes.
Using 6381 files for training.


W0000 00:00:1736857173.849509   12652 gpu_backend_lib.cc:579] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /home/ysh/miniconda3/envs/gpu-old/lib/python3.9/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/ysh/miniconda3/envs/gpu-old/lib/python3.9/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  /home/ysh/miniconda3/envs/gpu-old/lib/python3.9/site-packages/tensorflow/python/platform/../../cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.


INFO  Loading Validating Data.
Found 7976 files belonging to 2 classes.
Using 1595 files for validation.
INFO  Loading Testing Data.
Found 1995 files belonging to 2 classes.


In [9]:
for text_batch, label_batch in train_ds.take(1):
    print(f'Review: {text_batch.numpy()[0]}')
    for i in range(3):
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({class_names[label]})')

Review: b'interact n point cartesian plane denot point coordin integ inclus three point collinear say point vertic convex polygon word exist permut integ 1 n polygon convex vertic list give number n hide coordin point task guess permut ask multipl queri queri give khanh 4 integ j k either 1 2 j k three distinct indic 1 n inclus respons khanh tell 1 area triangl multipli 2 2 sign cross product two vector recal cross product vector vector b integ sign number 1 posit otherwis proven cross product obtain queri 0 ask 3 n note khanh fix coordin point chang answer queri need guess coordin permut equal 1 indic vertic list order'
Label : 0 (neg)
Label : 0 (neg)
Label : 0 (neg)


2025-01-14 20:19:34.657999: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-01-14 20:19:34.659029: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1" # 這是預訓練模型
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3" # 這是 BERT 前處理需要用的模型

In [11]:
debug(pre['Statement'][0])

DEBUG watch new seri squid game mashtali soroush decid hold squid game soroush agre host provid money winner prize mashtali becam front man player regist play game win great prize mashtali found huge winner prize go decid kill elimin player could take money evil mashtali go elimin player unroot tree n vertic everi player 2 special vertic one oper mashtali choos vertex v tree remain player find vertex w simpl path closest v player mashtali wonder minimum number oper perform remov everi player game take money sinc think money could solv problem ask help


In [12]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess) # 呼叫前處理模型
text_test = [pre['Statement'][0]] # 先輸入簡單的句子看看會變成什麼樣子
text_preprocessed = bert_preprocess_model(text_test)
print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_word_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [  101  3422  2047 14262  2072 26852  2208 16137 22893  3669  2061 13288]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:

bert_model = hub.KerasLayer(tfhub_handle_encoder)
debug('---')
bert_results = bert_model(text_preprocessed)
print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')


In [12]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)


In [13]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

epochs = 10
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')


In [14]:
classifier_model = build_classifier_model()

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)            

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [54]:
classifier_model.save_weights('model.st')

In [ ]:
debug([pre['isDP'][i] for i in list(pre['isDP'])].count(True))
debug(len(pre['isDP']))

In [ ]:
df = pd.read_csv('../data/filtered_test_data.csv')
df = df[['Statement', 'tags']]
df['isDP'] = ['dp' in i for i in df['tags'].tolist()]
df = df.drop('tags', axis = 1)
df

In [ ]:
now = df.to_dict()
n = len(now['Statement'])
cd(root)
cd('aclImdb')
cd ('test')
for tag in ['neg', 0], ['pos', 1]:
    cd(tag[0])
    for i in range(n):
        if now['isDP'][i] ^ tag[1]: continue;
        info(f'[{i} / {n}] Parsing Rows', end = '\r')
        write_to_file(f'{i}.txt', now['Statement'][i])
    info(f'[OK] Task Parse {tag[0]} Finished.')
    cd('..')
cd('../..')